In [3]:
import pandas as pd
import plotly.express as px  # (version 4.7.0)
import plotly.graph_objects as go

from jupyter_dash import JupyterDash
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

In [4]:
app = dash.Dash(__name__)

# ------------------------------------------------------------------------------
# Import and clean data (importing csv into pandas)
athletes_sn = pd.read_csv("athlete_events.csv")
noc_regions_sn = pd.read_csv("noc_regions.csv")

df_sn = pd.merge(left=athletes_sn,right=noc_regions_sn,how='left',on='NOC')
df_sn.drop_duplicates()
df_new_sn = df_sn.sort_values(by=['region', 'Year'])
df_new_sn = df_new_sn.groupby(['Season','region','Year'])['ID'].count().reset_index(name='count')

NOC_values_sn = df_new_sn['region'].unique()
year_values_sn = df_new_sn['Year'].unique()


medal_list_sn = ['Gold','Silver','Bronze']
df_medals_sn = df_sn[df_sn['Medal'].isin(medal_list_sn)]
df_medals_sn.head()
df_medals_sn['count'] = df_sn.groupby(['region','Season','Medal','Year'])['ID'].transform('count')




C:\Users\sruth\anaconda3\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [5]:
app.layout = html.Div(className = 'row', children=[
    html.H1("Region-Wise Participation in the Olympic Games", style={'text-align': 'center'}),
    html.Br(),
    html.H2("Summary of the dashboard:"),
    html.P("The Doughnut chart represents the participation of various regions in the Summer and Winter Olympics."),
    html.P("The barcharts display the number of medals won by each region for each year"),
    html.Br(),
    html.Div([
    html.Div([
    html.P("Choose a country: "),
    dcc.Dropdown(
                id='country_dropdown',
                options=[{'label': i, 'value': i} for i in NOC_values_sn],
                multi=False,
                value='Austria',
                style={'width': "50%"}
                )
    ],
    style={'width': '50%', 'display': 'inline-block'}),
    
   
    html.Div([
    html.P("Select a Year: "),
    dcc.Dropdown(
                id='year-dropdown',
                options = sorted([{'label': j, 'value': j} for j in year_values_sn], key = lambda w: w['label']),
                multi=False,
                value='1948',
                style={'width': "50%"}
                )
    ], style={'width': '49%', 'float': 'right', 'display': 'inline-block'})
    
    ], style={
        'borderBottom': 'thin lightgrey solid',
        'backgroundColor': 'rgb(250, 250, 250)',
        'padding': '10px 5px'
    }),
    
      html.Br(),

    
    html.Div(children=[
        dcc.Graph(id='first-chart',figure={},style={'text-align': 'center','display': 'inline-block'}),
        dcc.Graph(id='second-chart',figure={},style={'text-align': 'center','display': 'inline-block'})   
    ])

    ])


In [6]:
@app.callback(
    [Output(component_id='first-chart', component_property='figure'),
     Output(component_id='second-chart', component_property='figure')],
    [Input(component_id='country_dropdown', component_property='value'),
     Input(component_id='year-dropdown', component_property='value')]
    )
def update_graph(reg_chosen_sn,year_selected_sn):
    print(reg_chosen_sn,year_selected_sn)
    
    dff1_sn = df_new_sn.copy()
    filter1_sn = dff1_sn['region'].isin([reg_chosen_sn])
    filter2_sn = dff1_sn['Year'].isin([year_selected_sn])
    dff1_sn = dff1_sn[filter1_sn & filter2_sn]

       
    fig1_sn = px.pie(dff1_sn, values='count', names='Season', 
       title='Seasonal distribution for the chosen country', 
       #opacity=0.7,
       labels={'Season':'Season','count':'Participants'},
       color='Season',
       color_discrete_map={"Summer": "orangered" ,"Winter":"blue"},               
       hole=0.3)
    
    fig1_sn.update_traces(textposition='inside', textinfo='percent+label')
   
    fig1_sn.update_layout(title_font_size=17,title_x=0.5,uniformtext_minsize=8)
    
    
    dff2_sn = df_medals_sn.copy()
    filter3_sn = dff2_sn['region'].isin([reg_chosen_sn])
    filter4_sn = dff2_sn['Year'].isin([year_selected_sn])
    dff2_sn = dff2_sn[filter3_sn & filter4_sn]

    fig2_sn = px.bar(dff2_sn,
        x='Season',y='count',
        orientation='v',
        title='No of medals won in Summer and Winter',
        hover_name='Season',labels={'Season': 'Season', 'count': 'Medals Won'},
        facet_col = 'Medal',
        color='Season',
        color_discrete_map={"Summer": "orangered" ,"Winter":"blue"})
    
    
    fig2_sn.update_layout(title_font_size=17,title_x=0.5,uniformtext_minsize=8)
    fig2_sn.update_yaxes(showticklabels=False, visible=True)
    
   
    
    return fig1_sn,fig2_sn
    

In [ ]:
app.run_server()